## PBL(2): Pest Classification - template code for CNN

#### PBL(2) 프로젝트를 수행하기 위한 템플릿 코드입니다: CNN 을 활용한 해충 분류

1. https://agtechresearch.pythonanywhere.com/ 에 가입된 이메일과 비밀번호를 사용해주세요.

    (회원가입을 하지 않았다면, 위 사이트에 접속하여 회원가입해 주세요: 비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

In [1]:
project = "pestclassification"  # 수정하지 마세요
username = ""  # 회원가입 시 사용한 이메일아이디 (예시. abc@hello.com)
password = ""  # 비밀번호

리더보드 제출을 위한 기본 설정: 아래 코드를 실행해주세요.


In [2]:
import os
import urllib.request

if not os.path.exists("competition.py"):
    url = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/competition/competition.py"
    filename = "competition.py"
    urllib.request.urlretrieve(url, filename)

### 데이터셋 다운로드

아래 코드를 실행하여 데이터(dataset.zip)를 다운로드 받고, dataset 폴더에 압축을 해제합니다.

    - 학습용 데이터셋: trainset
    - problem set: problemset

데이터 다운로드와 압축 해제가 성공적으로 진행되었다면, 2개의 폴더와 2개의 csv 파일이 dataset 폴더 내부에 생성됨

* trainset 폴더: 학습용 데이터셋. 작물재배에 유해한 10종류의 해충 이미지들(aphids, armyworm, blisterbeetle, cicadellidae, cornborer, cricket, delicatula, limacodidae, miridae, viridis)
* problemset 폴더: 학습된 모델에 의하여 예측을 수행하여야 할 데이터셋 -> 리더보드 제출을 위한 problem

* problem.csv: 예측을 수행하여야 할 데이터셋 이미지들의 FilePath 포함
* submission.csv: 리더보드 서버 제출을 위한 파일 형식


In [3]:
!wget https://github.com/agtechresearch/LectureAlgorithm/raw/main/pestclassification/dataset.zip

!unzip dataset.zip -d dataset

In [3]:
from IPython.display import Image
import numpy as np
import pandas as pd
import torch
import warnings

# Data 경로 설정
DATA_DIR = "dataset"

# 경고 무시
warnings.filterwarnings("ignore")

In [4]:
# 데이터 디렉토리 설정: trainset, problemset
data_dir = os.path.join(DATA_DIR, 'trainset')
problem_dir = os.path.join(DATA_DIR, 'problemset')

### 학습용 데이터셋을 폴더로부터 읽어들이기 위해 ImageFolder 사용

 - 폴더에 위치한 이미지를 tensor 형식으로 변환
 - 폴더 내 위치한 이미지에 대하여 라벨링 기능 지원
 
 

In [6]:
Image(url='https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/img/folderstruct.png', width=150)

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# 이미지 전처리 정의
train_transform = transforms.Compose(
    [
        transforms.Resize((160, 160)),  # 이미지 크기를 (160, 160)로 조정
        transforms.RandomHorizontalFlip(0.5),  # 50% 확률로 랜덤하게 이미지 좌우 반전
        transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # 이미지를 정규화
    ]
)

# ImageFolder를 사용하여 학습용 데이터셋 dataset 생성
dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2)

In [6]:
# trainset의 클래스 정보 확인 (추후 학습된 모델에 의해 예측 결과가 숫자로 도출됨 -> 숫자 라벨을 클래스 이름으로 변환하여 사용하기 위함)
print(dataset.class_to_idx)

# 숫자 라벨을 클래스 이름으로 매핑하는 딕셔너리 생성
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

{'aphids': 0, 'armyworm': 1, 'blisterbeetle': 2, 'cicadellidae': 3, 'cornborer': 4, 'cricket': 5, 'delicatula': 6, 'limacodidae': 7, 'miridae': 8, 'viridis': 9}


### 데이터셋 중 일부 이미지들을 화면에 display

In [ ]:
import torchvision
import matplotlib.pyplot as plt

# 한 배치의 이미지 시각화 함수 (사이즈 조정 포함)
def imshow(img, labels, classes):
    img = img.numpy().transpose((1, 2, 0))
    plt.figure(figsize=(20, 20))  # 여기에서 플롯의 크기를 조정
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    # 이미지마다 클래스 레이블을 타이틀로 표시
    for i, label in enumerate(labels):
        x = (i % 8) * (img.shape[1] / 8) + (img.shape[1] / 16)
        y = (i // 8) * (img.shape[0] / 2) + 1  # 4 rows
        plt.text(
            x,
            y,
            classes[label],
            ha="center",
            va="top",
            color="white",
            fontsize=12,
            backgroundcolor="black",
        )
    plt.show()


# 데이터 로더에서 한 배치 가져오기
dataiter = iter(data_loader)
images, labels = next(dataiter)

# 이미지 그리드 만들기
img_grid = torchvision.utils.make_grid(images, nrow=8)  # 8개의 이미지를 한 줄에 표시

# 이미지와 레이블 시각화
imshow(img_grid, labels, dataset.classes)

### 주어진 데이터셋을 학습(train_data)과 검증(val_data)로 분할하여 학습이 진행되는 중 validation loss, validation accuracy 를 함께 화면에 출력

In [8]:
from torch.utils.data import random_split

ratio = 0.8  # 학습셋(train set)의 비율을 설정합니다.

train_size = int(ratio * len(dataset))
val_size = len(dataset) - train_size
print(f"total: {len(dataset)}\ntrain_size: {train_size}\nval_size: {val_size}")

# random_split으로 8:2의 비율로 train / validation 세트를 분할합니다.
train_data, val_data = random_split(dataset, [train_size, val_size])

total: 9990
train_size: 7992
val_size: 1998


In [9]:
# DataLoader 인스턴스 train_loader 생성
# batch_size 의 경우 일반적으로는 16 혹은 32 로 설정
# num_workers 는 대개 컴퓨터의 코어 수와 동일하게 설정

batch_size = 16  # batch_size 지정
num_workers = 2  # Thread 숫자 지정 (병렬 처리에 활용할 쓰레드 숫자 지정)

train_loader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
val_loader = DataLoader(
    val_data, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

### CNN (합성곱 신경망) 모델의 정의: Convolution layer, ReLu(activation), Max-pooling 들의 조합과 마지막 단계에서는 FC layers 들로 구성

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Device 설정 (cuda:0 혹은 cpu)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.sequential = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Linear(256 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 128)
        self.classifier = nn.Linear(128, 10)

    def forward(self, x):
        x = self.sequential(x)
        x = torch.flatten(x, 1) #  평탄화(일자의 벡터로 만듦)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.classifier(x)
        return x

cpu


### 정의된 CNN 모델을 생성하고 device 에 할당(GPU/CUDA or CPU)

In [11]:
# 모델 생성
model = CNNModel()

# 모델을 device 에 올림 (cuda:0 혹은 cpu)
model.to(device)
model

CNNModel(
  (sequential): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=6400, out_features=512, bias=True)
  (fc2): Linear(in_feat

### 생성된 CNN 모델의 구조(shape)를 화면에 출력: 각 레이어에서 출력되는 output shape 과 더불어 파라미터의 사이즈, 모델의 용량 등을 확인

In [12]:
!pip install torchsummary -q

from torchsummary import summary

summary(model, (3, 160, 160))  # -1은 배치사이즈(얘는 모름)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 160, 160]             448
              ReLU-2         [-1, 16, 160, 160]               0
         MaxPool2d-3           [-1, 16, 80, 80]               0
            Conv2d-4           [-1, 32, 80, 80]           4,640
              ReLU-5           [-1, 32, 80, 80]               0
         MaxPool2d-6           [-1, 32, 40, 40]               0
            Conv2d-7           [-1, 64, 40, 40]          18,496
              ReLU-8           [-1, 64, 40, 40]               0
         MaxPool2d-9           [-1, 64, 20, 20]               0
           Conv2d-10          [-1, 128, 20, 20]          73,856
             ReLU-11          [-1, 128, 20, 20]               0
        MaxPool2d-12          [-1, 128, 10, 10]               0
           Conv2d-13          [-1, 256, 10, 10]         295,168
             ReLU-14          [-1, 256,

### 손실함수 및 옵티마이저 정의/설정

In [13]:
# Cross Entropy Loss 정의
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 설정: model.paramters()와 learning_rate 설정
optimizer = optim.Adam(model.parameters(), lr=0.0001)

### 학습 및 검증을 위한 절차를 하나의 클래스로 정의

In [14]:
from tqdm import tqdm


def fit(model, data_loader, loss_fn, optimizer, device, phase="train"):
    if phase == "train":
        # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
        model.train()
    else:
        # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
        model.eval()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, leave=False)

    # mini-batch 학습을 시작합니다.
    for img, lbl in prograss_bar:
        # image, label 데이터를 device에 올립니다.
        img, lbl = img.to(device), lbl.to(device)

        optimizer.zero_grad()
        # 누적 Gradient를 초기화 합니다.
        with torch.set_grad_enabled(phase == "train"):

            # Forward Propagation을 진행하여 결과를 얻습니다.
            output = model(img)

            # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
            loss = loss_fn(output, lbl)

            if phase == "train":
                # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
                loss.backward()

                # 계산된 Gradient를 업데이트 합니다.
                optimizer.step()

        # output 의 뉴런별 확률 값을 sparse vector 로 변환합니다.
        pred = output.argmax(axis=1)

        # 정답 개수를 카운트 합니다.
        corr += (lbl == pred).sum().item()

        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item()

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader), acc

### 학습을 진행: 학습이 진행되는 동안 매 epoch 마다 loss, accuracy 를 화면에 출력
 - 학습이 진행되는 과정 중 validation loss 가 가장 낮은 때의 모델 파라미터들을 파일(CNNModel.pth)로 저장 

In [ ]:
import time

# 최대 Epoch을 지정합니다.
num_epochs = 10

min_loss = np.inf

STATE_DICT_PATH = "CNNModel.pth"

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    start = time.time()
    train_loss, train_acc = fit(
        model, train_loader, loss_fn, optimizer, device, phase="train"
    )

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = fit(
        model, val_loader, loss_fn, optimizer, device, phase="eval"
    )

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(
            f"[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!"
        )
        min_loss = val_loss
        torch.save(model.state_dict(), STATE_DICT_PATH)

    time_elapsed = time.time() - start
    # Epoch 별 결과를 출력합니다.
    print(
        f"[Epoch{epoch+1:02d}] time: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s \t loss: {train_loss:.5f}, acc: {train_acc:.5f} | val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}"
    )

 - 위에서 진행되었던 학습 epoch 과정 중 가장 낮은 validation loss 를 나타낸 모델의 파라미터들을 로드

In [16]:
# 모델에 저장한 가중치를 로드합니다.
model.load_state_dict(torch.load(STATE_DICT_PATH))

<All keys matched successfully>

In [ ]:
# 최종 검증 손실(validation loss)와 검증 정확도(validation accuracy)를 산출합니다.
final_loss, final_acc = fit(
    model, val_loader, loss_fn, optimizer, device, phase="eval"
)
print(f"\nevaluation loss: {final_loss:.5f}, evaluation accuracy: {final_acc:.5f}")

## Problemset 이미지들을 불러오기
 - 학습이 완료된 CNN 모델을 사용하여 problemset 이미지들에 대한 분류/예측 수행

In [18]:
problem = pd.read_csv(os.path.join(DATA_DIR, "problem.csv"))
problem.head()

,FilePath
0,./dataset/problemset/001.jpg
1,./dataset/problemset/002.jpg
2,./dataset/problemset/003.jpg
3,./dataset/problemset/004.jpg
4,./dataset/problemset/005.jpg


In [19]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]  # 'FilePath' 열에서 이미지 경로 가져오기
        image = Image.open(img_path).convert("RGB")  # 이미지를 RGB로 로드
        #image = Image.open(img_path).convert("L")  # 이미지를 흑백으로 로드

        if self.transform:
            image = self.transform(image)

        return image


# 이미지를 불러올 때 적용할 전처리 정의: resize, to tensor
problem_transform = transforms.Compose(
    [
        transforms.Resize((160, 160)),  # 이미지 크기를 (28, 28)로 조정
        transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
    ]
)

# 커스텀 데이터셋 인스턴스 생성
custom_dataset = CustomImageDataset(dataframe=problem, transform=problem_transform)

# DataLoader 인스턴스 생성
problem_loader = DataLoader(custom_dataset, batch_size=1, shuffle=False)

## Problem set 문제에 대한 예측 및 리더보드 결과 제출

In [20]:
predictions = []

# 검증모드 진입
model.eval()

with torch.no_grad():
    # loss 초기화
    running_loss = 0
    # 정확도 계산
    running_acc = 0
    for x in problem_loader:
        x = x.to(device)

        y_hat = model(x)
        label = y_hat.argmax(dim=1).detach().item()
        predictions.append(label)

In [21]:
# 숫자 라벨을 클래스 이름으로 변환
your_answer = [idx_to_class[l] for l in predictions]

- 아래 제출 프로세스가 느리다고 중지 후 다시 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능: 최대 100회까지 가능)
- 제출에 성공할 경우, "제출에 성공하였습니다"의 메세지와 함께 제출 결과 accuracy 가 화면에 출력됩니다.
- 제출결과는 또한 [대회 페이지(리더보드 서버)](https://agtechresearch.pythonanywhere.com/competitions/pestclassification/)의 `리더보드` 와 `제출` 탭에서 확인할 수 있습니다.

In [22]:
import competition

# 리더보드 서버 제출을 위한 파일 생성(예측 결과 업데이트)
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["Label"] = your_answer

# 예측 결과 화면에 출력 후 제출
display(submission)
competition.submit(project, username, password, submission)

,FilePath,Label
0,./dataset/problemset/001.jpg,aphids
1,./dataset/problemset/002.jpg,blisterbeetle
2,./dataset/problemset/003.jpg,armyworm
3,./dataset/problemset/004.jpg,delicatula
4,./dataset/problemset/005.jpg,viridis
...,...,...
195,./dataset/problemset/196.jpg,viridis
196,./dataset/problemset/197.jpg,cornborer
197,./dataset/problemset/198.jpg,viridis
198,./dataset/problemset/199.jpg,limacodidae


아이디:  abc@abc.co.kr
파일명:  submissions/20240613-050129-submission.csv
[제출에 성공하였습니다]
제출 결과: 0.245
